In [ ]:
!pwd

/content


In [ ]:
path="/content/drive/MyDrive/Project/ImagePro"

In [ ]:
import tensorflow as tf
import keras
import cv2
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [ ]:
files=os.listdir(path)

In [ ]:
files.sort()
print(files)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [ ]:
image_array=[]
label_array=[]

In [ ]:
#loop thru each file in files
for i in tqdm(range(len(files))):
    #list of image in each folder
    sub_file=os.listdir(path+"/"+files[i])

    #loop thru each sub folder
    for j in range(len(sub_file)):
        file_path=path+"/"+files[i]+"/"+sub_file[j]
        image=cv2.imread(file_path)
        image=cv2.resize(image,(96,96))
        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image_array.append(image)
        label_array.append(i)

100%|██████████| 26/26 [08:26<00:00, 19.48s/it]


In [ ]:
image_array=np.array(image_array)
label_array=np.array(label_array,dtype="float")

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(image_array,label_array,test_size=0.15)
#xtrain will have 85% images and xtest will have 15% images


In [ ]:
del image_array, label_array

In [ ]:
import gc
gc.collect()

0

In [ ]:
from keras import layers,callbacks,utils,applications,optimizers
from keras.models import Sequential,Model,load_model

In [ ]:
model=Sequential()
#add pretrained model to sequential model. adding EfficientNetB0-TRANSFER LEARNING used
pretrained_model=tf.keras.applications.EfficientNetB0(input_shape=(96,96,3),include_top=False)

16705208/16705208 [==============================] - 1s 0us/step


In [ ]:
model.add(pretrained_model)

In [ ]:
#add pooling to model
model.add(layers.GlobalAveragePooling2D())

In [ ]:
#add dropout to model
#we add dropout to increase accuracy by reducing overfitting
model.add(layers.Dropout(0.3))

In [ ]:
#add dense layer as output
model.add(layers.Dense(1))

In [ ]:
model.build(input_shape=(None,96,96,3))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional  (None, 3, 3, 1280)        4049571   
 )                                                               
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 4050852 (15.45 MB)
Trainable params: 4008829 (15.29 MB)
Non-trainable params: 42023 (164.16 KB)
_________________________________________________________________


In [ ]:
#compile model
model.compile(optimizer="adam",loss="mae",metrics=["mae"])

In [ ]:
#create checkpoint to save best accuracy model
ckp_path="trained_model/model"
model_checkpoint=tf.keras.callbacks.ModelCheckpoint(
filepath=ckp_path,
monitor="val_mae",
mode="auto",
save_best_only=True,
save_weights_only=True)

In [ ]:
#create learning rate reducer to reduce lr when accuracy does not improve
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(
factor=0.9, monitor="val_mae", mode="auto",cooldown=0,patience=5,verbose=1,min_lr=1e-6)

In [ ]:
#start training the model
Epochs=100
Batch_Size=32

In [ ]:
history=model.fit(X_train,
                  Y_train,
                  validation_data=(X_test,Y_test),
                  batch_size=Batch_Size,
                  epochs=Epochs,
                  callbacks=[model_checkpoint,reduce_lr]
                 )

Epoch 1/100
316/316 [==============================] - 68s 81ms/step - loss: 3.0734 - mae: 3.0734 - val_loss: 2.2279 - val_mae: 2.2279 - lr: 0.0010
Epoch 2/100
316/316 [==============================] - 22s 68ms/step - loss: 1.2692 - mae: 1.2692 - val_loss: 0.8391 - val_mae: 0.8391 - lr: 0.0010
Epoch 3/100
316/316 [==============================] - 21s 65ms/step - loss: 0.8956 - mae: 0.8956 - val_loss: 1.0708 - val_mae: 1.0708 - lr: 0.0010
Epoch 4/100
316/316 [==============================] - 22s 68ms/step - loss: 0.7369 - mae: 0.7369 - val_loss: 0.4661 - val_mae: 0.4661 - lr: 0.0010
Epoch 5/100
316/316 [==============================] - 21s 66ms/step - loss: 0.6536 - mae: 0.6536 - val_loss: 0.5882 - val_mae: 0.5882 - lr: 0.0010
Epoch 6/100
316/316 [==============================] - 21s 66ms/step - loss: 0.5898 - mae: 0.5898 - val_loss: 0.8362 - val_mae: 0.8362 - lr: 0.0010
Epoch 7/100
316/316 [==============================] - 21s 66ms/step - loss: 0.5428 - mae: 0.5428 - val_loss: 0.

In [ ]:
#after training load best model
model.load_weights(ckp_path)

In [ ]:
#convert model to tensorflow lite model
converter=tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model=converter.convert()

In [ ]:
#save model
with open("model.tflite","wb") as f:
  f.write(tflite_model)

In [ ]:
#to see prediction result on test dataset
prediction_val=model.predict(X_test,batch_size=32)
#print 1st 10 values
print(prediction_val[:10])

56/56 [==============================] - 3s 15ms/step
[[19.12621  ]
 [ 9.013281 ]
 [21.109003 ]
 [10.045968 ]
 [23.889221 ]
 [18.090654 ]
 [16.019152 ]
 [22.91921  ]
 [14.908449 ]
 [ 0.9970049]]


In [ ]:
#print 1st 10 values of ytest
print(Y_test[:10])

[19.  9. 21. 10. 24. 18. 16. 23. 15.  1.]


In [ ]:
# Evaluate the model on the test set
accuracy = model.evaluate(X_test, Y_test, verbose=1)

# Print the accuracy in terms of MAE
print("Test MAE:", accuracy[1])

56/56 [==============================] - 1s 15ms/step - loss: 0.1064 - mae: 0.1064
Test MAE: 0.10644066333770752
